In [3]:
import requests
import pandas as pd
import math

In [2]:
restaurant = pd.read_csv('restaurants.csv')
restaurant.drop(columns=['score', 'ratings', 'price_range', 'position'], inplace=True)
restaurant['state'] = restaurant['full_address'].str.split(', ').str[2]
restaurant = restaurant[restaurant['state'] == 'VA']
restaurant.head()

,id,name,category,full_address,zip_code,lat,lng,state
17438,17439,Ruby Tuesday (3150 Kiln Creek Parkway),"Burgers, American, Sandwiches","3150 Kiln Creek Pkwy, Yorktown, VA, 23693",23693,37.117247,-76.464591,VA
17439,17440,Asian grill,"Asian, Chinese, Sushi, Asian Fusion, South Asian","12442 Warwick Blvd, Newport News, VA, 23606",23606,37.071450,-76.492640,VA
17440,17441,Donutz On A Stick Ice Cream and Grill,"Dessert: Other, Ice Cream + Frozen Yogurt, Bur...","2809 Hampton Hwy, Yorktown, VA, 23693",23693,37.102905,-76.418884,VA
17441,17442,Domoishi - Hampton,"Asian, Asian Fusion, Wings","2870 Kilgore Avenue, Hampton, VA, 23666",23666,37.042260,-76.390800,VA
17442,17443,1865 Brewing Company,"Alcohol, Black-owned, American, Coffee and Tea...","9 South Mallory Street, Hampton, VA, 23663",23663,37.018910,-76.321540,VA


In [3]:
menu = pd.read_csv('restaurant-menus.csv')
menu.drop_duplicates
menu['dishes'] = menu['name'] + ": " + menu['price']
menu.drop(columns=['category', 'name', 'description', 'price'], inplace=True)
menu = menu.groupby('restaurant_id')['dishes'].apply(list)
menu_df = pd.DataFrame({'id': menu.index, 'dishes': menu.values})
menu_df.head()

,id,dishes
0,1,"[Extra Large Meat Lovers: 15.99 USD, Extra Lar..."
1,2,"[J’ Ti`’z Smoothie: 5.49 USD, Ti`’z Fruity Tha..."
2,3,"[Chicken Bacon Ranch Cheesesteak: 14.38 USD, P..."
3,4,[Chicken Bacon Artichoke - Keto Friendly (Baki...
4,5,"[Pork Chop Rice with Gravy Plate: 7.0 USD, Ful..."


In [4]:
whole_restaurant = restaurant.merge(menu_df, on='id')
whole_restaurant.head()

,id,name,category,full_address,zip_code,lat,lng,state,dishes
0,17439,Ruby Tuesday (3150 Kiln Creek Parkway),"Burgers, American, Sandwiches","3150 Kiln Creek Pkwy, Yorktown, VA, 23693",23693,37.117247,-76.464591,VA,[Paper Napkins (Airlaid) - 1000 ct Case: 35.19...
1,17440,Asian grill,"Asian, Chinese, Sushi, Asian Fusion, South Asian","12442 Warwick Blvd, Newport News, VA, 23606",23606,37.071450,-76.492640,VA,"[C16 General Tso's Chicken: 11.25 USD, Chicken..."
2,17441,Donutz On A Stick Ice Cream and Grill,"Dessert: Other, Ice Cream + Frozen Yogurt, Bur...","2809 Hampton Hwy, Yorktown, VA, 23693",23693,37.102905,-76.418884,VA,"[Spicy Sausage: 8.99 USD, Vanilla Milk Shake: ..."
3,17442,Domoishi - Hampton,"Asian, Asian Fusion, Wings","2870 Kilgore Avenue, Hampton, VA, 23666",23666,37.042260,-76.390800,VA,"[Poké Bowl: 13.95 USD, Ramen Noodle: 10.95 USD..."
4,17443,1865 Brewing Company,"Alcohol, Black-owned, American, Coffee and Tea...","9 South Mallory Street, Hampton, VA, 23663",23663,37.018910,-76.321540,VA,"[Margarita: 13.0 USD, Big island chicken : 16...."


In [15]:
api_key = 'YpUUvTIrwy3zbuhAwzHUDYVx8Bv2xuqQD_EVhs9eLMDNVpJSWDepEs2shca3wWOeexghkB9vaiicy-pNJNEtxJOnlqNYHM9LVrV17zC1Ijzh0uWMtRBb-oYM-uRBY3Yx'
headers = {'Authorization': 'Bearer ' + api_key}
search_api_url = 'https://api.yelp.com/v3/businesses/matches'
params = {}
count = 0
id_list = []
restaurant_id_list = []
while count <= 500:
    for i in range(4900, len(whole_restaurant)):
        try :
            add_list = whole_restaurant['full_address'][i].split(', ')
        except:
            print('cant split')
            continue
        params['name'] = whole_restaurant['name'][i]
        params['address1'] = add_list[0]
        params['city'] = add_list[1]
        params['state'] = add_list[2]
        params['zip_code'] = add_list[3]
        params['country'] = 'US'
        lat = whole_restaurant['lat'][i]
        lng = whole_restaurant['lng'][i]
        response = requests.get(search_api_url, headers=headers, params=params)
        try:   
            new_lat = response.json()['businesses'][0]['coordinates']['latitude']
            new_lng = response.json()['businesses'][0]['coordinates']['longitude']
        except:
            print('error!')
            continue
        if math.isclose(lat, new_lat, abs_tol=0.001) & math.isclose(lng, new_lng, abs_tol=0.001):
            id_list.append(response.json()['businesses'][0]['id'])
            restaurant_id_list.append(whole_restaurant['id'][i])
            count += 1
            print(count)
        else:
            print('wrong!')
        # if count > 500:
        #         break



1
2
3
4
5
6
7
error!
8
9
10
11
error!
12
error!
13
14
15
16
17
18
19
20
21
error!
22
23
24
25
error!
26
27
error!
error!
28
29
30
31
32
33
34
error!
35
36
37
38
39
error!
40
41
42
43
44
45
error!
46
error!
47
48
error!
49
error!
error!
50
51
52
wrong!
53
error!
54
55
56
wrong!
57
wrong!
error!
58
59
wrong!
60
wrong!
61
error!
error!
error!
error!
62
error!
63
error!
64
65
66
error!
67
68
69
70
71
72
73
74
error!
error!
error!
75
76
77
78
79
80
81
82
error!
83
84
85
86
87
88
89
90
wrong!
91
92
error!
error!
93
94
wrong!
error!
95
96
error!
97
98
error!
error!
99
error!
100
101
error!
102
103
104
105
error!
106
107
error!
108
error!
109
110
111
112
113
114
115
wrong!
116
117
118
error!
119
120
error!
error!
121
error!
error!
122
wrong!
123
124
125
126
127
wrong!
128
129
130
131
132
133
error!
error!
134
135
136
137
138
139
wrong!
140
error!
141
142
wrong!
143
error!
error!
144
145
146
error!
error!
147
error!
error!
error!
148
error!
149
error!
error!
150
151
152
error!
error!
error!
153

In [16]:
old = pd.read_csv('data.csv')
contain = restaurant[restaurant['id'].isin(restaurant_id_list)]
contain.insert(loc=0, column='uniq_id', value=id_list)
new = pd.concat([old, contain])
new
# new.drop(columns=['Unnamed: 0'], inplace=True)
# old
# contain

,uniq_id,id,name,category,full_address,zip_code,lat,lng,state
0,l0GoDj7bRDpRPOzT3B8yjQ,17439,Ruby Tuesday (3150 Kiln Creek Parkway),"Burgers, American, Sandwiches","3150 Kiln Creek Pkwy, Yorktown, VA, 23693",23693,37.1172473,-76.4645911,VA
1,y4oAop23TUE-sHPDdZ0I0Q,17440,Asian grill,"Asian, Chinese, Sushi, Asian Fusion, South Asian","12442 Warwick Blvd, Newport News, VA, 23606",23606,37.07145,-76.49264,VA
2,MMlYeZegOajGPNs6RRXGZQ,17441,Donutz On A Stick Ice Cream and Grill,"Dessert: Other, Ice Cream + Frozen Yogurt, Bur...","2809 Hampton Hwy, Yorktown, VA, 23693",23693,37.102905,-76.418884,VA
3,aMUyRUmk_QH6hggdUhLrig,17442,Domoishi - Hampton,"Asian, Asian Fusion, Wings","2870 Kilgore Avenue, Hampton, VA, 23666",23666,37.04226,-76.3908,VA
4,i5-EJKci6mtUY35v3Ovp6Q,17443,1865 Brewing Company,"Alcohol, Black-owned, American, Coffee and Tea...","9 South Mallory Street, Hampton, VA, 23663",23663,37.01891,-76.32154,VA
...,...,...,...,...,...,...,...,...,...
29228,7S4EIHnRxMIvfBtuvSacLA,29229,Logan's Roadhouse 373 (3174 Linden Drive),"Burgers, American, Sandwiches, Family Meals","3174 Linden Drive, Bristol, VA, 24202",24202,36.630454,-82.125221,VA
29230,EeZNQMZF4uKuvCD2nvV0qg,29231,Buffalo Wild Wings (240 Falls Blvd),"wings, American, Chicken","240 Falls Blvd, Bristol, VA, 24202",24202,36.62932,-82.14609,VA
29231,A6_wSqgCG4eU2UnJ1UKjzw,29232,Sheetz - 2415 Lee Highway (599),"American, Comfort Food, Deli, Desserts, Dinner...","2415 Lee Highway, Bristol, VA, 24202",24202,36.62881,-82.14902,VA
29233,bGHOsNNJu_DhJwnjYk2p_Q,29234,Bojangles (1020 Old Airport Road),"American, Sandwich, Salads, Family Meals","1020 Old Airport Road, Bristol, VA, 24201",24201,36.629129,-82.122025,VA


In [11]:
class root:
    def __init__(self, zip_code, id_list = []):
        self.left = None
        self.right = None
        self.zipcode = zip_code
        self.id = id_list
    
    def insert(self, zip_code, id_list = []):
        if self.zipcode == zip_code:
            return
        elif zip_code < self.zipcode:
            if self.left:
                self.left.insert(zip_code, id_list)
            else:
                self.left = root(zip_code, id_list)
        else:
            if self.right:
                self.right.insert(zip_code, id_list)
            else:
                self.right = root(zip_code, id_list)

    def insert_id(self, zip_code, id):
        if self.zipcode == zip_code:
            self.id.append(id)
        elif self.zipcode > zip_code:
            self.left.insert_id(zip_code, id)
        else:
            self.right.insert_id(zip_code, id)

    def find(self, zip_code):
        if self.zipcode == zip_code:
            return self.id
        elif self.zipcode > zip_code:
            return self.left.find(zip_code)
        else:
            return self.right.find(zip_code)